# Crawling Whoscored Web Site
- https://www.whoscored.com/

In [1]:
# import package
import pandas as pd
import time
from selenium import webdriver

In [2]:
api_delay_term = 5

In [3]:
def str_to_zero(point):
    return 0 if point == "-" else point

In [4]:
# cawling league & team id
# parameter : team id
# return : league teams & team id
def crawling_league_teams(team_id):
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver =  webdriver.PhantomJS()
    driver.get(url)
    time.sleep(api_delay_term)
    team_names = driver.find_elements_by_css_selector("#teams option")
    
    df = pd.DataFrame(columns=["team_id","team_name"])
    
    for team_name in team_names:
        name = team_name.text 
        team_id = team_name.get_attribute("value").split("/")[2]
        tmp_dict = {"team_id":team_id, "team_name":name }
        df.loc[len(df)] = tmp_dict
    return df

In [5]:
# team players summary list crawling
# parameter : team id
# return : pandas DataFrame : player summary features
def crawling_player_summary(team_id):
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver =  webdriver.PhantomJS()
    driver.get(url)
    time.sleep(api_delay_term)
    elements = driver.find_elements_by_css_selector("#player-table-statistics-body tr")
    
    df = pd.DataFrame(columns=[
            'player_number', 'flag', 'name', 'age', 'position', 'tall', 'weight', 'full_time', 'half_time', 'mins', 
            'goals', 'asists', 'yel', 'red', 'spg', 'ps', 'motm', 'aw', 'rating',
        ])
    
    for element in elements:

        games = element.find_elements_by_css_selector("td")[5].text
        games = games.split("(")
        full_time, half_time = games[0], 0
        if len(games) > 1 :
            half_time = games[1].replace(")","")
        else :
            half_time = 0

        tmp_dict = { 
            "player_number": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "flag": element.find_elements_by_css_selector("td")[1].find_elements_by_css_selector("span")[0].get_attribute("class").split("-")[2],
            "name": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("a")[0].text, 
            "age": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("span")[0].text, 
            "position": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("span")[1].text[1:], 
            "tall": element.find_elements_by_css_selector("td")[3].text,
            "weight": element.find_elements_by_css_selector("td")[4].text, 
            "full_time": full_time,
            "half_time": half_time,
            "mins": str_to_zero(element.find_elements_by_css_selector("td")[6].text),
            "goals": str_to_zero(element.find_elements_by_css_selector("td")[7].text),
            "asists": str_to_zero(element.find_elements_by_css_selector("td")[8].text),
            "yel": str_to_zero(element.find_elements_by_css_selector("td")[9].text),
            "red": str_to_zero(element.find_elements_by_css_selector("td")[10].text),
            "spg": str_to_zero(element.find_elements_by_css_selector("td")[11].text),
            "ps": str_to_zero(element.find_elements_by_css_selector("td")[12].text),
            "aw": str_to_zero(element.find_elements_by_css_selector("td")[13].text),
            "motm": str_to_zero(element.find_elements_by_css_selector("td")[14].text),
            "rating": str_to_zero(element.find_elements_by_css_selector("td")[15].text),
        }
        df.loc[len(df)] = tmp_dict
    return df

In [6]:
# team players defensive list crawling
# parameter : team id
# return : pandas DataFrame : player defensive features
def crawling_player_defensive(team_id):
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.PhantomJS()
    driver.get(url)
    time.sleep(api_delay_term)
    driver.find_elements_by_css_selector("#team-squad-stats-options .in-squad-detailed-view")[0].find_element_by_css_selector("a").click()
    time.sleep(api_delay_term)
    elements = driver.find_elements_by_css_selector("#team-squad-stats-defensive #player-table-statistics-body tr")

    df = pd.DataFrame(columns=[
            "player_number", "tackles", "inter", "fouls", "offsides", "clear", "drb", "blocks", "owng"
        ])

    for element in elements:
       
        tmp_dict = {
            "player_number": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "tackles": str_to_zero(element.find_elements_by_css_selector("td")[7].text), 
            "inter": str_to_zero(element.find_elements_by_css_selector("td")[8].text), 
            "fouls": str_to_zero(element.find_elements_by_css_selector("td")[9].text),
            "offsides": str_to_zero(element.find_elements_by_css_selector("td")[10].text),
            "clear": str_to_zero(element.find_elements_by_css_selector("td")[11].text),
            "drb": str_to_zero(element.find_elements_by_css_selector("td")[12].text),
            "blocks": str_to_zero(element.find_elements_by_css_selector("td")[13].text),
            "owng": str_to_zero(element.find_elements_by_css_selector("td")[14].text),
        }
        
        df.loc[len(df)] = tmp_dict
    return df

In [7]:
# team players offensive list crawling
# parameter : team id
# return : pandas DataFrame : player offensive features
def crawling_player_offensive(team_id):
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.PhantomJS()
    driver.get(url)
    time.sleep(api_delay_term)
    driver.find_elements_by_css_selector("#team-squad-stats-options .in-squad-detailed-view")[1].find_element_by_css_selector("a").click()
    time.sleep(api_delay_term)
    elements = driver.find_elements_by_css_selector("#statistics-table-offensive #player-table-statistics-body tr")

    df = pd.DataFrame(columns=[
            "player_number", "keyp", "fouled", "off", "disp", "unstch"
        ])

    for element in elements:
       
        tmp_dict = {
            "player_number": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "keyp": str_to_zero(element.find_elements_by_css_selector("td")[10].text), 
            "fouled": str_to_zero(element.find_elements_by_css_selector("td")[12].text), 
            "off": str_to_zero(element.find_elements_by_css_selector("td")[13].text),
            "disp": str_to_zero(element.find_elements_by_css_selector("td")[14].text),
            "unstch": str_to_zero(element.find_elements_by_css_selector("td")[15].text),
        }
        
        df.loc[len(df)] = tmp_dict
    return df

In [8]:
# team players passing list crawling
# parameter : team id
# return : pandas DataFrame : player passing features
def crawling_player_passing(team_id):
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.PhantomJS()
    driver.get(url)
    time.sleep(api_delay_term)
    driver.find_elements_by_css_selector("#team-squad-stats-options .in-squad-detailed-view")[2].find_element_by_css_selector("a").click()
    time.sleep(api_delay_term)
    elements = driver.find_elements_by_css_selector("#statistics-table-passing #player-table-statistics-body tr")

    df = pd.DataFrame(columns=[
            "player_number", "avgp", "ps", "crosses", "longb", "thrb"
        ])

    for element in elements:
       
        tmp_dict = {
            "player_number": element.find_elements_by_css_selector("td")[2].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "avgp": str_to_zero(element.find_elements_by_css_selector("td")[8].text), 
            "ps": str_to_zero(element.find_elements_by_css_selector("td")[9].text), 
            "crosses": str_to_zero(element.find_elements_by_css_selector("td")[10].text),
            "longb": str_to_zero(element.find_elements_by_css_selector("td")[11].text),
            "thrb": str_to_zero(element.find_elements_by_css_selector("td")[12].text),
        }
        
        df.loc[len(df)] = tmp_dict
    return df

In [9]:
# 13 : Arsenal : Premier League
# 87 : Juventus : Serie A
# 52 : Real Madrid : La Liga
# 37 : Bayern Munich : Bundesliga

# crawling league team data

# LaLiga = crawling_league_teams(52)
# SerieA = crawling_league_teams(87)
# PremierLeague = crawling_league_teams(13)
# Bundesliga = crawling_league_teams(37)

# LaLiga.to_csv("./league/LaLiga.csv", index=False)
# SerieA.to_csv("./league/SerieA.csv", index=False)
# PremierLeague.to_csv("./league/PremierLeague.csv", index=False)
# Bundesliga.to_csv("./league/Bundesliga.csv", index=False)

In [15]:
# crawling & merge player datas
def make_players_info(team_id, team_name):
    print(team_id, team_name)
    player_summary_df = crawling_player_summary(team_id)
    print("player_summary")
    player_defensive_df = crawling_player_defensive(team_id)
    print("player_defensive")
    player_offensive_df = crawling_player_offensive(team_id)
    print("player_offensive")
    player_passing_df = crawling_player_passing(team_id)
    print("player_passing")
    
    print("player crawling DONE")
    
    sd = player_summary_df.merge(player_defensive_df, on="player_number")
    sdo = sd.merge(player_offensive_df, on="player_number")
    sdop = sdo.merge(player_passing_df, on="player_number")
    
    sdop["team_name"] = team_name
    
    print(len(sdop))
    
    return sdop

In [11]:
def save_player_info(league):
    df = pd.read_csv("./league/" + league + ".csv")
    for idx, row in df.iterrows():
        players_df = make_players_info(row.team_id, row.team_name)
        players_df.to_csv("./player/" + league + "/" + row.team_name + ".csv")
        print(row.team_name) 
    print(league + "Done")

In [13]:
# 팀별로 저장

# PremierLeague, SerieA, LaLiga, Bundesliga
df = pd.read_csv("./league/PremierLeague.csv")

df

,team_id,team_name
0,13,Arsenal
1,24,Aston Villa
2,183,Bournemouth
3,15,Chelsea
4,162,Crystal Palace
5,31,Everton
6,14,Leicester
7,26,Liverpool
8,167,Manchester City
9,32,Manchester United


In [ ]:
# PremierLeague, SerieA, LaLiga, Bundesliga

league = "PremierLeague"
team_id = 30
team_name = "Tottenham"

start_time = time.time()
player_df = make_players_info(team_id, team_name)
print("excute_time : {time}".format(time=(time.time() - start_time)))

30 Tottenham
player_summary
player_defensive
player_offensive


In [31]:
player_df["league"] = league

In [32]:
path = "./player/" + league + "/" + team_name + ".csv"
player_df.to_csv(path, index=False)

In [33]:
tmp_df = pd.read_csv(path)
tmp_df.head()

,player_number,flag,name,age,position,tall,weight,full_time,half_time,mins,...,off,disp,unstch,avgp,ps_y,crosses,longb,thrb,team_name,league
0,25832,gb,Troy Deeney,27,FW,180,76,36,2,3293,...,0.5,1.6,1.8,1.5,34.0,60.9,0.0,0.9,Watford,PremierLeague
1,122945,nl,Nathan Aké,21,"D(L),DMC",180,75,20,4,1875,...,0.0,0.1,0.8,0.4,25.4,72.7,0.2,1.3,Watford,PremierLeague
2,10133,br,Heurelho Gomes,35,GK,191,91,38,0,3397,...,0.0,0.0,0.0,0.0,25.2,46.4,0.0,9.1,Watford,PremierLeague
3,33590,fr,Etienne Capoue,27,"D(C),M(C)",189,80,33,0,2822,...,0.1,1.4,1.3,0.7,46.8,79.2,0.2,3.4,Watford,PremierLeague
4,31376,ng,Odion Ighalo,27,"AM(C),FW",188,70,36,1,3153,...,0.7,2.2,2.8,1.0,19.5,75.9,0.1,0.2,Watford,PremierLeague
